# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [24]:
import os
from glob import glob
import dask.dataframe as dd
import pandas as pd

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
PRICE_DATA



'../../05_src/data/prices/'

In [15]:
os.getenv("PRICE_DATA")

'../../05_src/data/prices/'

In [16]:
os.path.join(os.getenv("PRICE_DATA"), "**/*.parquet")

'../../05_src/data/prices/**/*.parquet'

In [56]:
parquet_files = glob(os.path.join(os.getenv("PRICE_DATA"), "**/*.parquet"), recursive = True)
parquet_files

['../../05_src/data/prices\\ACIO\\ACIO_2019\\part.0.parquet',
 '../../05_src/data/prices\\ACIO\\ACIO_2019\\part.1.parquet',
 '../../05_src/data/prices\\ACIO\\ACIO_2020\\part.0.parquet',
 '../../05_src/data/prices\\ACIO\\ACIO_2020\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\

In [19]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ACIO,datetime64[ns],float64,float64,float64,float64,float64,int64,string,int32
ACN,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


In [21]:
len(dd_px), len(parquet_files)

(321724, 2779)

In [22]:
dd_px.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
ticker,,,,,,,,,
ACIO,2019-07-10,25.120001,25.160000,25.040001,25.115999,24.822029,170000.0,ACIO.csv,2019
ACIO,2019-07-11,25.049999,25.110001,24.990000,25.110001,24.816103,385700.0,ACIO.csv,2019
ACIO,2019-07-12,25.150000,25.219999,25.150000,25.219999,24.924814,229600.0,ACIO.csv,2019
ACIO,2019-07-15,25.160000,25.200001,25.160000,25.166000,24.871447,292600.0,ACIO.csv,2019
ACIO,2019-07-16,25.035000,25.129999,25.035000,25.124001,24.829937,167100.0,ACIO.csv,2019
...,...,...,...,...,...,...,...,...,...
ZIXI,1989-12-22,7.066667,7.333333,7.066667,7.266667,7.121057,5200.0,ZIXI.csv,1989
ZIXI,1989-12-26,7.266667,7.466667,7.266667,7.333333,7.186388,30700.0,ZIXI.csv,1989
ZIXI,1989-12-27,7.333333,7.333333,7.266667,7.333333,7.186388,9700.0,ZIXI.csv,1989


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [34]:
# Write your code below.

dd_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag_1 = x['Adj Close'].shift(1)), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

In [35]:
dd_feat = dd_feat.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [36]:
df_ticker = dd_feat.compute()
df_ticker

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns
ticker,,,,,,,,,,,,
ACIO,2019-07-10,25.120001,25.160000,25.040001,25.115999,24.822029,170000.0,ACIO.csv,2019,NaN,NaN,NaN
ACIO,2019-07-11,25.049999,25.110001,24.990000,25.110001,24.816103,385700.0,ACIO.csv,2019,25.115999,24.822029,-0.000239
ACIO,2019-07-12,25.150000,25.219999,25.150000,25.219999,24.924814,229600.0,ACIO.csv,2019,25.110001,24.816103,0.004381
ACIO,2019-07-15,25.160000,25.200001,25.160000,25.166000,24.871447,292600.0,ACIO.csv,2019,25.219999,24.924814,-0.002141
ACIO,2019-07-16,25.035000,25.129999,25.035000,25.124001,24.829937,167100.0,ACIO.csv,2019,25.166000,24.871447,-0.001669
...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.060000,4.530000,3.880000,4.510000,4.510000,1668500.0,ZIXI.csv,2020,7.066667,6.925064,-0.361792
ZIXI,2020-03-27,4.490000,4.710000,4.100000,4.600000,4.600000,1146800.0,ZIXI.csv,2020,7.266667,7.121057,-0.366973
ZIXI,2020-03-30,4.830000,4.870000,4.440000,4.640000,4.640000,1212000.0,ZIXI.csv,2020,7.333333,7.186388,-0.367273


In [32]:
type(df_ticker)

pandas.core.frame.DataFrame

In [55]:
df_ticker[["returns","Volume"]]



,returns,Volume
ticker,,
ACIO,NaN,170000.0
ACIO,-0.000239,385700.0
ACIO,0.004381,229600.0
ACIO,-0.002141,292600.0
ACIO,-0.001669,167100.0
...,...,...
ZIXI,-0.361792,1668500.0
ZIXI,-0.366973,1146800.0
ZIXI,-0.367273,1212000.0


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [59]:
# Write your code below.
df_ticker["moving_average"] = (
    df_ticker.groupby("ticker")["returns"]
            .rolling(10)
            .mean()
            .reset_index(level=0,drop=True)
)


In [62]:
df_ticker.tail(20)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,moving_average
ticker,,,,,,,,,,,,,
ZIXI,2020-03-05,8.05,8.32,7.94,8.03,8.03,674600.0,ZIXI.csv,2020,7.266667,7.121057,0.105046,0.125830
ZIXI,2020-03-06,7.81,7.82,7.32,7.49,7.49,716900.0,ZIXI.csv,2020,7.266667,7.121057,0.030734,0.110497
ZIXI,2020-03-09,6.79,7.07,6.48,6.73,6.73,706600.0,ZIXI.csv,2020,6.933333,6.794402,-0.029327,0.086359
ZIXI,2020-03-10,6.97,7.03,6.74,6.91,6.91,527200.0,ZIXI.csv,2020,6.666667,6.533080,0.036500,0.073725
ZIXI,2020-03-11,6.73,6.85,5.90,5.99,5.99,994700.0,ZIXI.csv,2020,6.533333,6.402418,-0.083163,0.045271
ZIXI,2020-03-12,5.48,5.56,4.88,4.88,4.88,770000.0,ZIXI.csv,2020,6.533333,6.402418,-0.253061,0.009874
ZIXI,2020-03-13,5.21,5.39,4.58,4.73,4.73,903100.0,ZIXI.csv,2020,6.800000,6.663741,-0.304412,-0.026103
ZIXI,2020-03-16,4.50,4.52,3.99,4.24,4.24,995800.0,ZIXI.csv,2020,6.666667,6.533080,-0.364000,-0.071655
ZIXI,2020-03-17,4.23,4.23,3.86,3.92,3.92,1017900.0,ZIXI.csv,2020,6.800000,6.663741,-0.423529,-0.118743


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.